# Artificial Neural Network

### Importing the libraries

In [1]:
import numpy as np
import pandas as pd
import tensorflow as tf
from sklearn.model_selection import GridSearchCV
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from scikeras.wrappers import KerasRegressor

## Part 1 - Data Preprocessing

### Importing the dataset

In [2]:
dataset = pd.read_excel('Dataset3.xlsx')
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 4 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Charge_type        546 non-null    object 
 1   Charge_size        546 non-null    float64
 2   Standoff_distance  546 non-null    float64
 3   Incident_pressure  546 non-null    float64
dtypes: float64(3), object(1)
memory usage: 17.2+ KB


In [3]:
# convert categorical variable into dummy variables
dataset = pd.get_dummies(dataset, columns=['Charge_type'])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 546 entries, 0 to 545
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Charge_size        546 non-null    float64
 1   Standoff_distance  546 non-null    float64
 2   Incident_pressure  546 non-null    float64
 3   Charge_type_CompB  546 non-null    uint8  
 4   Charge_type_TNT    546 non-null    uint8  
dtypes: float64(3), uint8(2)
memory usage: 14.0 KB


In [4]:
y = dataset['Incident_pressure']
X = dataset.drop('Incident_pressure', axis=1)
print(X.shape, y.shape)

(546, 4) (546,)


In [5]:
# convert to numpy array
X = np.array(X)
y = np.array(y)

### Splitting the dataset into the Training set and Test set

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

### Hyperparameter tuning - layers, neurons, activation function

In [7]:
from tensorflow.keras.wrappers.scikit_learn import KerasRegressor as KR
import math
def FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes):
    layers = []
    
    nodes_increment = (last_layer_nodes - first_layer_nodes)/ (n_layers-1)
    nodes = first_layer_nodes
    for i in range(1, n_layers+1):
        layers.append(math.ceil(nodes))
        nodes = nodes + nodes_increment
    
    return layers

In [8]:
from tensorflow.keras.optimizers import Adam
def create_model(n_layers, first_layer_nodes, last_layer_nodes, activation_func):
    model = Sequential()
    n_nodes = FindLayerNodesLinear(n_layers, first_layer_nodes, last_layer_nodes)
    for i in range(1, n_layers):
        if i==1:
            model.add(Dense(units = first_layer_nodes,  input_shape=(X_train.shape[1],), activation=activation_func))
        else:
            model.add(Dense(n_nodes[i-1], activation=activation_func))
            
    #Finally, the output layer should have a single node in binary classification
    model.add(Dense(1, activation='linear'))
    opt = Adam(learning_rate=0.001)
    model.compile(optimizer = opt, loss = 'mean_squared_error', metrics = ['mae'])
    return model

##Wrap model into scikit-learn
model = KR(build_fn=create_model, epochs = 500, batch_size = 50)

C:\Users\Chathura Gamage\AppData\Local\Temp\ipykernel_9444\2480470563.py:18: DeprecationWarning: KerasRegressor is deprecated, use Sci-Keras (https://github.com/adriangb/scikeras) instead. See https://www.adriangb.com/scikeras/stable/migration.html for help migrating.
  model = KR(build_fn=create_model, epochs = 500, batch_size = 50)


In [9]:
activation_funcs = ['relu', 'softplus', 'leaky_relu'] 
param_grid = dict(n_layers=[2,3], first_layer_nodes = [4,5,6], last_layer_nodes = [2,3,4],  activation_func = activation_funcs)
grid = GridSearchCV(estimator = model, param_grid = param_grid,n_jobs=-1, cv=3, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/500
8/8 [==============================] - 0s 2ms/step - loss: 1337330.3750 - mae: 442.0508
Epoch 2/500
8/8 [==============================] - 0s 2ms/step - loss: 1336952.3750 - mae: 441.6393
Epoch 3/500
8/8 [==============================] - 0s 2ms/step - loss: 1336577.3750 - mae: 441.2387
Epoch 4/500
8/8 [==============================] - 0s 2ms/step - loss: 1336214.5000 - mae: 440.8464
Epoch 5/500
8/8 [==============================] - 0s 3ms/step - loss: 1335881.0000 - mae: 440.4448
Epoch 6/500
8/8 [==============================] - 0s 3ms/step - loss: 1335535.6250 - mae: 440.0442
Epoch 7/500
8/8 [==============================] - 0s 2ms/step - loss: 1335202.6250 - mae: 439.6343
Epoch 8/500
8/8 [==============================] - 0s 2ms/step - loss: 1334892.0000 - mae: 439.2038
Epoch 9/500
8/8 [==============================] - 0s 2ms/step - loss: 1334572.6250 - mae: 438.7547
Epoch 10/500
8/8 [==============================] - 0s 2ms/step - loss: 1334251.6250 - mae: 438.2687

### Hyperparameter tuning - batch size, epoch, optimizer, learning rate

In [10]:
# Function to create model, required for KerasClassifier
def create_model(optimizer):
 # create model
 model = Sequential()
 model.add(Dense(units=6, input_shape=(X_train.shape[1],), activation='softplus'))
 model.add(Dense(units=5, activation='softplus'))
 model.add(Dense(units=4, activation='softplus'))
 model.add(Dense(units=1, activation='linear'))
 # Compile model
 model.compile(optimizer = optimizer, loss = 'mean_squared_error', metrics = ['mae'])
 return model

In [11]:
# create model
model = KerasRegressor(model=create_model)

# define the grid search parameters
batch_size = [50, 60, 80]
epochs = [300, 400, 500]
optimizer = ['RMSprop', 'Adam', 'Adamax', 'Nadam']
learn_rate = [0.001, 0.01, 0.1]

# gridsearch
param_grid = dict(batch_size=batch_size, epochs=epochs, model__optimizer=optimizer, optimizer__learning_rate=learn_rate)
grid = GridSearchCV(estimator=model, param_grid=param_grid, n_jobs=-1, cv=3, scoring = 'r2')
grid_result = grid.fit(X_train, y_train)

# summarize results
print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))
means = grid_result.cv_results_['mean_test_score']
stds = grid_result.cv_results_['std_test_score']
params = grid_result.cv_results_['params']
for mean, stdev, param in zip(means, stds, params):
    print("%f (%f) with: %r" % (mean, stdev, param))

Epoch 1/500
7/7 [==============================] - 1s 2ms/step - loss: 1333307.0000 - mae: 440.8825
Epoch 2/500
7/7 [==============================] - 0s 2ms/step - loss: 1333124.0000 - mae: 440.7425
Epoch 3/500
7/7 [==============================] - 0s 2ms/step - loss: 1332896.3750 - mae: 440.5765
Epoch 4/500
7/7 [==============================] - 0s 3ms/step - loss: 1332603.7500 - mae: 440.3639
Epoch 5/500
7/7 [==============================] - 0s 3ms/step - loss: 1332295.5000 - mae: 440.1154
Epoch 6/500
7/7 [==============================] - 0s 3ms/step - loss: 1331926.5000 - mae: 439.8488
Epoch 7/500
7/7 [==============================] - 0s 2ms/step - loss: 1331516.6250 - mae: 439.4888
Epoch 8/500
7/7 [==============================] - 0s 2ms/step - loss: 1331015.2500 - mae: 439.0458
Epoch 9/500
7/7 [==============================] - 0s 2ms/step - loss: 1330472.1250 - mae: 438.5599
Epoch 10/500
7/7 [==============================] - 0s 2ms/step - loss: 1329855.3750 - mae: 438.0184